In [1]:
import os
import json

print("FILES IN THIS DIRECTORY")
print(os.listdir(os.getcwd()))
print(os.getcwd())

# config.json 파일 읽기
with open("config.json", 'r') as f:
    config = json.load(f)


FILES IN THIS DIRECTORY
['.bashrc', '.bash_logout', '.profile', 'work', 'spark-streaming-kafka-0-10_2.12-3.2.4.jar', 'dataset1.csv', 'test.ipynb', 'spark-sql-kafka-0-10_2.12-3.2.4.jar', '.ipython', '.cache', '.npm', '.bash_history', '.local', '.ipynb_checkpoints', 'config.json', '.ivy2', '.jupyter', '.conda', '.config', '.wget-hsts']
/home/jovyan


In [2]:
from pyspark.sql import SparkSession
import pandas as pd

jar_urls = ",".join(config['JAR_URLS'])

spark = (
    SparkSession.builder.master("spark://10.42.1.48:7077")
    .config("spark.driver.host", "10.42.2.75")
    .config("spark.driver.port", "39337")
    .config("spark.executor.instances", '3') # 3 -> 추천
    .config("spark.executor.cores", '16') # 16 -> 추천
    .config("spark.executor.memory", '24G') # "24G" -> 추천
    .config("spark.defaul.parallelism", 3*16*3) # 3(익스큐터 수) x 16(익스큐터 코어 수) x 2 ~ 3 -> 추천
    .config("spark.sql.shuffle.partitions", 3*16*3) # 3(익스큐터 수) x 16(익스큐터 코어 수) x 2 ~ 3 -> 추천
    .config("spark.scheduler.mode", "FAIR") # 선택
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .config("spark.jars", jar_urls)  # JAR 파일 포함 (라이브러리 필요하면)
    .appName("hobit") # 아무거나 반드시 넣어야함.
    .getOrCreate()
)


sc = spark.sparkContext
sc
  
  

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 09:28:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/05 09:28:45 WARN Utils: Service 'sparkDriver' could not bind on port 39337. Attempting port 39338.
24/05/05 09:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/05 09:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/05 09:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


<SparkContext master=spark://10.42.1.48:7077 appName=hobit>

In [3]:
print("Current Spark configuration:")
for key, value in sorted(sc._conf.getAll(), key=lambda x: x[0]):
    print(f"{key} = {value}")

Current Spark configuration:
spark.app.id = app-20240505092845-0026
spark.app.initial.jar.urls = https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-client-api/3.3.1/hadoop-client-api-3.3.1.jar,https://repo1.maven.org/maven2/commons-logging/commons-logging/1.1.3/commons-logging-1.1.3.jar,https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.2.4/spark-sql-kafka-0-10_2.12-3.2.4.jar,https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.2.2/mongo-spark-connector_2.12-10.2.2.jar,https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.2.4/spark-token-provider-kafka-0-10_2.12-3.2.4.jar,https://repo1.maven.org/maven2/com/google/code/findbugs/jsr305/3.0.0/jsr305-3.0.0.jar,https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.6.2/commons-pool2-2.6.2.jar,https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.2.4/spark-streaming-kafka-0-10_2.12-3.2.4.jar,https://repo1.maven.org/

In [6]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "my-cluster-kafka-bootstrap.kafka.svc:9092") \
  .option("subscribe", "my-topic") \
  .load()
  
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
df.show()



Py4JJavaError: An error occurred while calling o108.showString.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.UninterruptibleThreadRunner.runUninterruptibly(UninterruptibleThreadRunner.scala:46)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.fetchTopicPartitions(KafkaOffsetReaderConsumer.scala:129)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.fetchPartitionOffsets(KafkaOffsetReaderConsumer.scala:150)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.getOffsetRangesFromUnresolvedOffsets(KafkaOffsetReaderConsumer.scala:422)
	at org.apache.spark.sql.kafka010.KafkaRelation.buildScan(KafkaRelation.scala:67)
	at org.apache.spark.sql.execution.datasources.DataSourceStrategy$.apply(DataSourceStrategy.scala:339)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$

In [ ]:
print(os.environ.get("SPARK_HOME"))

In [5]:
spark.stop()